# Setup Environment

In [ ]:
!pip install geomloss[full]==0.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 5.6 MB/s 
     |████████████████████████████████| 213 kB 78.9 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 137 kB 83.5 MB/s 
     |████████████████████████████████| 2.8 MB 67.0 MB/s 
     |████████████████████████████████| 92 kB 939 kB/s 
     |████████████████████████████████| 361 kB 78.8 MB/s 
     |████████████████████████████████| 115 kB 76.9 MB/s 
     |████████████████████████████████| 4.7 MB 51.0 MB/s 
     |████████████████████████████████| 3.2 MB 65.2 MB/s 
     |████████████████████████████████| 121 kB 93.9 MB/s 
     |████████████████████████████████| 90 kB 11.6 MB/s 
     |████████████████████████████████| 133 kB 96.7 MB/s 
     |████████████████████████████████| 100 kB 11.8 MB/s 
     |████████████████████████████████| 1.1 MB 70.8 MB/s 
     |████████████████████████████████

In [ ]:
import os
import os.path

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from scipy.spatial import distance
from scipy.spatial.distance import directed_hausdorff

import torch
from geomloss import SamplesLoss

[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


#Import 652 dataset for TP-53 mutation prediciton

You can find the zip file at: https://drive.google.com/file/d/1Q4KvkxSl8cimAZXxxmvU1WcdI_2Wa1bg/view?usp=sharing

In [ ]:
#Recommended to store datasets on google drive as they are then loaded quicker in colab
!unzip "/content/drive/MyDrive/TCGA_BRCA_20x_512_512_ShuffleNet_652_slides.zip" -d "/content/"

Archive:  /content/drive/MyDrive/datasets/20x_512_512_noresize_ShuffleNet_fixed.zip
   creating: /content/ShuffleNet/
  inflating: /content/ShuffleNet/TCGA-3C-AALJ_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-3C-AALK_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-5T-A9QA_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SB_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SE_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SH_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SI_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SK_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SM_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SN_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SP_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A1-A0SQ_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A2-A04P_ShuffleNet.npz  
  inflating: /content/ShuffleNet/TCGA-A2-A04Q_ShuffleNet.npz  


Use this to generate a new list of slide IDs

In [ ]:
slide_IDs_652 = []
for filename in os.listdir('/content/ShuffleNet/'):
  slide_IDs_652.append(filename[:-15])
np.save('/content/slide_IDs_652.npy',slide_IDs_652)

652


Use this to load a pre-exisitng list of slide IDs

In [ ]:
slide_IDs_652 = np.load('/content/drive/MyDrive/Hist-MMD/data/MMD_matrix/slide_IDs_652.npy')

# Import 1051 slide dataset for surival analysis

You can find the zip file at: https://drive.google.com/file/d/1VhFjyFtiu4UwnbImGl-0iLJPOmp1FQ5c/view?usp=sharing

In [ ]:
!unzip "/content/drive/MyDrive/TCGA_BRCA_20x_512_512_ShuffleNet_1051_slides.zip"

Archive:  /content/drive/MyDrive/dataset/ShuffleNet_20x_512x512_noresize_1054patients.zip
   creating: ShuffleNet/
  inflating: ShuffleNet/TCGA-3C-AALI_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-3C-AALJ_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-3C-AALK_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-4H-AAAK_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-5L-AAT0_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-5L-AAT1_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-5T-A9QA_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SB_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SD_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SE_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SF_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SH_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SI_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SJ_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SK_ShuffleNet.npz  
  inflating: ShuffleNet/TCGA-A1-A0SM_ShuffleNet.npz  
  inflating: ShuffleN

Use this to generate a new list of slide IDs

In [ ]:
slide_IDs_1051 = []
for filename in os.listdir("/content/ShuffleNet/"):
  slide_IDs_1051.append(filename[:-15]) #-15 to remove "_ShuffleNet.npz" from file name
np.save("/content/slide_IDs_1051.npy", slide_IDs_1051)

Use this to load a pre-exisitng list of slide IDs

In [ ]:
slide_IDs_1051 = np.load("/content/drive/MyDrive/Hist-MMD/data/MMD_matrix/slide_IDs_1051.npy")

# Distance Matrix Function

In [ ]:
class DistanceMatrix:
  def __init__(self, datasetPath, fileTail, numFeatures, slides, dictionaryKey=None):
     """
        Parameters
        ----------
        datasetPath : str
            The path to the folder with the .npz files for the dataset
        fileTail : str
            the text in a filename after the slide ID e.g. for 'TCGA-A2-A4RY_ShuffleNet.npz' that would be '_ShuffleNet.npz'
        numFeatures : int
            The number of features of each patch represnation
        slides : array
            The IDs of the slides in the dataset
        dictionaryKey: str
            The key used to access the patch feature represnetion from the .npz file
    """

    self.datasetPath = datasetPath
    self.fileTail = fileTail
    self.dictionaryKey = dictionaryKey

    self.slides = slides
    
    self.numFeatures = numFeatures

  def MMD_distance(self, distribution, blur, start, filename, savePath, D=None):
    """ Calculates the MMD distance for the dataset.

        Parameters
        ----------
        distribution : str
            The distribution used for the dataset e.g. 'gaussian', 'laplacian' etc.
        blur : float
            the standard deviation of the kernel
        start : int
            If some rows of the matrx are already computed from a previous run then can set start row to the last compiled row
        filename : str
            Filename of distance matrix
        savePath : str
            As colab times out every 10 rows the kernel progress is saved to this destination
        D: N * N array
            Can pass in a partially computed kernel from a previous run to carry on without resetting progress
    """

    lossTesnor = SamplesLoss(loss=distribution, blur=blur, backend="tensorized")
    lossOnline = SamplesLoss(loss=distribution, blur=blur, backend="online")

    elements = len(self.slides)

    if (D is None):
      D = np.zeros([elements,elements], dtype=np.float32)

    rowsDone = start

    for i in range(start,elements):

      slide = self.datasetPath + self.slides[i] + self.fileTail
      if(self.dictionaryKey):
        X = np.load(slide,allow_pickle=True)[self.dictionaryKey]
      else:
        X = np.load(slide,allow_pickle=True)

      x = torch.tensor(X, device=torch.device('cuda:0'))

      for j in range(i+1,elements):
        
        if(D[i,j] != 0):
          continue

        slide = self.datasetPath + self.slides[j] + self.fileTail
        if(self.dictionaryKey):
          Y = np.load(slide,allow_pickle=True)[self.dictionaryKey]
        else:
          Y = np.load(slide,allow_pickle=True)

        y = torch.tensor(Y, device=torch.device('cuda:0'))

        if(y.shape[0] > x.shape[0]):
          size = y.shape[0]
        else:
          size = x.shape[0]

        if(size <= 5000):
          L = lossTesnor(x,y)
        else:
          L = lossOnline(x,y)

        D[i,j] = L.item()

        del y,L
      del x

      rowsDone += 1
      print('Rows Done:'+str(rowsDone))
      if(rowsDone % 10 == 0):
        print('Saving: ' + str(rowsDone))
        np.save(savePath + str(rowsDone) + '.npy', D)
        print('Done saving')
    return self.save_full_D(D,filename,savePath)

  def save_full_D(self,upper_D,filename,location):
    """ Fills in the lower part of a matrix that only has the upper part computed.

        Parameters
        ----------
        upper_D : N * N array
            Kernel matrix that only has upper portion computed
        filename : stry
            the filename of the kernel to be saved
        savePath : str
            Where to save the kernel
    """
    D_cpy = upper_D.copy()
    D_cpy = D_cpy + D_cpy.T - np.diag(np.diag(D_cpy))

    path = location + filename + '.npy'
    np.save(path,D_cpy)

    return D_cpy


# MMD

In [ ]:
#Turns off warnings for GeomLoss
import pykeops
pykeops.set_verbose(False)

In [ ]:
#Folder where progress on D matrix is saved
!mkdir matrices

In [ ]:
calcD = DistanceMatrix(
                        datasetPath='/content/ShuffleNet/',
                        fileTail='_ShuffleNet.npz',
                        numFeatures=1024,
                        slides = slide_IDs_1051,
                        dictionaryKey='feat'
                      )

D = calcD.MMD_distance(
                        distribution = 'gaussian',
                        blur=10.0,
                        start=0,
                        filename='D_1051_blur_10',
                        savePath='/content/matrices/',
                        D=None
                      )

print(D)

Rows Done:1
Rows Done:2
Rows Done:3
Rows Done:4
Rows Done:5
Rows Done:6
Rows Done:7
Rows Done:8
Rows Done:9
Rows Done:10
Saving: 10
Done saving
Rows Done:11
Rows Done:12
Rows Done:13
Rows Done:14
Rows Done:15
Rows Done:16
Rows Done:17
Rows Done:18
Rows Done:19
Rows Done:20
Saving: 20
Done saving
Rows Done:21
Rows Done:22
Rows Done:23
Rows Done:24
Rows Done:25
Rows Done:26
Rows Done:27
Rows Done:28
Rows Done:29
Rows Done:30
Saving: 30
Done saving
Rows Done:31
Rows Done:32
Rows Done:33
Rows Done:34
Rows Done:35
Rows Done:36
Rows Done:37
Rows Done:38
Rows Done:39
Rows Done:40
Saving: 40
Done saving
Rows Done:41
Rows Done:42
Rows Done:43
Rows Done:44
Rows Done:45
Rows Done:46
Rows Done:47
Rows Done:48
Rows Done:49
Rows Done:50
Saving: 50
Done saving
Rows Done:51
Rows Done:52
Rows Done:53
Rows Done:54
Rows Done:55
Rows Done:56
Rows Done:57
Rows Done:58
Rows Done:59
Rows Done:60
Saving: 60
Done saving
Rows Done:61
Rows Done:62
Rows Done:63
Rows Done:64
Rows Done:65
Rows Done:66
Rows Done:67
